In [4]:
import numpy as np
import pandas as pd
import requests

In [14]:
#sql_query = "SELECT TOP 100 sobject_id, star_id, teff,e_teff,logg,e_logg, fe_h, e_fe_h FROM galah_dr3.main_star WHERE sobject_id < 160000000000000 and logg < 2.0" #this was the original ADQL query used by brent.miszalski@mq.edu.au
sql_query = "SELECT sobject_id FROM galah_dr3.main_star"
#Query GALAH DR3 catalogue using DataCentral API
api_url = 'https://datacentral.org.au/api/services/query/'
qdata = {'title' : 'get all sobject_ids of GALAH objects', #give a meaningful name
         'notes' : '', #ok if null
         'sql' : sql_query,
         'run_async' : False,
         'email'  : 'prav.nj@gmail.com'}
post = requests.post(api_url,data=qdata).json()
resp = requests.get(post['url']).json() #Python stores this in memory as a dict object
df = pd.DataFrame(resp['result']['data'],columns=resp['result']['columns']) #write to a df

In [17]:
df #this method restricts the query return to be 1000 records so we can't even get the full list of GALAH objects this way

,sobject_id
0,181222002601125
1,181222002601126
2,181222002601128
3,181222002601129
4,181222002601130
...,...
994,181222004601317
995,181222004601318
996,181222004601319
997,181222004601320


In [1]:
import astropy.io.fits as pyfits
import matplotlib.pyplot as plt
import glob

In [2]:
def read_spectra(sobject_id):
    fits_files = [[],[],[],[]]
    for each_ccd in [1,2,3,4]:
        fits_files[each_ccd-1] = glob.glob('C:/Users/pravn/Dropbox/masters/python-code/galah-file-read-test/data/GALAH_DR3/spectra/SPECTRA/'+str(sobject_id)+str(each_ccd)+'.fits') #this is reading fits files from file and not downloading directly 

    spectrum = dict()
    for each_ccd in [1,2,3,4]:
        if fits_files[each_ccd-1]!=[]:
            fits = pyfits.open(fits_files[each_ccd-1][0])
            
            # Extension 0: Reduced spectrum
            # Extension 1: Relative error spectrum
            # Extension 4: Normalised spectrum, NB: cut for CCD4

            # Extract wavelength grid for the reduced spectrum
            start_wavelength = fits[0].header["CRVAL1"]
            dispersion       = fits[0].header["CDELT1"]
            nr_pixels        = fits[0].header["NAXIS1"]
            reference_pixel  = fits[0].header["CRPIX1"]
            if reference_pixel == 0:
                reference_pixel = 1
            spectrum['wave_red_'+str(each_ccd)] = ((np.arange(0,nr_pixels)--reference_pixel+1)*dispersion+start_wavelength)

            # Extract wavelength grid for the normalised spectrum
            start_wavelength = fits[4].header["CRVAL1"]
            dispersion       = fits[4].header["CDELT1"]
            nr_pixels        = fits[4].header["NAXIS1"]
            reference_pixel  = fits[4].header["CRPIX1"]
            if reference_pixel == 0:
                reference_pixel=1
            spectrum['wave_norm_'+str(each_ccd)] = ((np.arange(0,nr_pixels)--reference_pixel+1)*dispersion+start_wavelength)

            # Extract flux and flux error of reduced spectrum
            spectrum['sob_red_'+str(each_ccd)]  = np.array(fits[0].data)
            spectrum['uob_red_'+str(each_ccd)]  = np.array(fits[0].data * fits[1].data)

            # Extract flux and flux error of reduced spectrum
            spectrum['sob_norm_'+str(each_ccd)] = np.array(fits[4].data)
            if each_ccd != 4:
                spectrum['uob_norm_'+str(each_ccd)] = np.array(fits[4].data * fits[1].data)
            else:
                # for normalised error of CCD4, only used appropriate parts of error spectrum
                spectrum['uob_norm_4'] = np.array(fits[4].data * (fits[1].data)[-len(spectrum['sob_norm_4']):])

            fits.close()
        else:
            spectrum['wave_red_'+str(each_ccd)] = []
            spectrum['wave_norm_'+str(each_ccd)] = []
            spectrum['sob_red_'+str(each_ccd)] = []
            spectrum['sob_norm_'+str(each_ccd)] = []
            spectrum['uob_red_'+str(each_ccd)] = []
            spectrum['uob_norm_'+str(each_ccd)] = []
    
    spectrum['wave_red'] = np.concatenate(([spectrum['wave_red_'+str(each_ccd)] for each_ccd in [1,2,3,4]]))
    spectrum['wave_norm'] = np.concatenate(([spectrum['wave_norm_'+str(each_ccd)] for each_ccd in [1,2,3,4]]))
    spectrum['sob_red'] = np.concatenate(([spectrum['sob_red_'+str(each_ccd)] for each_ccd in [1,2,3,4]]))
    spectrum['sob_norm'] = np.concatenate(([spectrum['sob_norm_'+str(each_ccd)] for each_ccd in [1,2,3,4]]))
    spectrum['uob_red'] = np.concatenate(([spectrum['uob_red_'+str(each_ccd)] for each_ccd in [1,2,3,4]]))
    spectrum['uob_norm'] = np.concatenate(([spectrum['uob_norm_'+str(each_ccd)] for each_ccd in [1,2,3,4]]))

    return spectrum 

In [5]:
read_spectra(170418003701205)

{'wave_red_1': array([4713.76881153, 4713.81483838, 4713.86086524, ..., 4902.15673828,
        4902.20276514, 4902.248792  ]),
 'wave_norm_1': array([4713.8050855 , 4713.85111272, 4713.89713993, ..., 4902.19446196,
        4902.24048917, 4902.28651638]),
 'sob_red_1': array([2864.1804, 2788.2244, 2733.7148, ..., 2276.3108, 2142.5562,
        1513.7109], dtype=float32),
 'uob_red_1': array([42.527172, 42.120983, 41.778976, ..., 39.948   , 39.730858,
        36.392746], dtype=float32),
 'sob_norm_1': array([0.94293747, 0.9179472 , 0.90001797, ..., 0.85457646, 0.8043833 ,
        0.56830734]),
 'uob_norm_1': array([0.01400068, 0.01386719, 0.01375485, ..., 0.01499735, 0.01491622,
        0.01366329]),
 'wave_red_2': array([5649.1850958 , 5649.2398019 , 5649.294508  , ..., 5873.09716797,
        5873.15187407, 5873.20658017]),
 'wave_norm_2': array([5649.2285681 , 5649.28327463, 5649.33798115, ..., 5873.14236335,
        5873.19706987, 5873.2517764 ]),
 'sob_red_2': array([5520.9805, 5534.6